# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
#query_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    #'address={0}&key={1}').format(target_city, gkey)
file = "../output_data/cities.csv"
cities_df = pd.read_csv(file)
cities_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
#url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
url = "https://maps.googleapis.com/maps/api/geocode/json?address="
query_url = url + "portland" + "&key=" + g_key
testerz = requests.get(query_url).json()
#print(json.dumps(testerz, indent=4))
#Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
# Fill NaN values and convert to float
# rating = airport_df["Airport Rating"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
humidity = cities_df['Humidity%'].astype(int)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heatmap_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
cities_df.head()

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere
0,calama,CL,-22.47,-68.93,46.40,15,0,9.17,South
1,cape town,ZA,-33.93,18.42,55.00,93,73,10.29,South
2,albany,US,42.60,-73.97,80.01,76,87,3.83,North
3,ushuaia,AR,-54.80,-68.30,33.80,96,75,13.87,South
4,udachnyy,RU,66.42,112.40,59.00,58,75,6.71,North


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
hotel_df = cities_df.loc[
    (cities_df["Max Temp(F)"]>70) &
    (cities_df["Max Temp(F)"]<80) &
    (cities_df["Windspeed(MPH)"]<10) &
    (cities_df["Cloudiness%"]==0)]
hotel_df = hotel_df.reindex(columns=hotel_df.columns.tolist() + ["Hotel Name"])
hotel_df

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
13,noumea,NC,-22.28,166.46,78.80,47,0,9.17,South,NaN
18,lagoa,PT,39.05,-27.98,70.30,69,0,6.80,North,NaN
83,laguna,US,38.42,-121.42,79.00,50,0,5.01,North,NaN
103,tupik,RU,54.43,119.93,74.26,54,0,5.17,North,NaN
173,westport,US,41.14,-73.36,78.01,91,0,8.61,North,NaN
180,airai,TL,-8.93,125.41,79.41,36,0,2.91,South,NaN
196,ribeira grande,PT,38.52,-28.70,70.20,69,0,7.90,North,NaN
202,marrakesh,MA,31.63,-8.01,71.60,64,0,3.36,North,NaN
215,chardara,KZ,41.25,67.97,73.51,33,0,8.41,North,NaN
261,sao miguel do araguaia,BR,-13.28,-50.16,74.37,37,0,2.35,South,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
locales = (hotel_df[['Latitude','Longitude']])

In [10]:
for index, row in locales.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location":f"{row[0]},{row[1]}",
        "radius":5000,
        "type":"lodging",
        "key":g_key
    }
    response = requests.get(url,params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
    except (KeyError,IndexError):
        pass
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
13,noumea,NC,-22.28,166.46,78.80,47,0,9.17,South,Hôtel Le Lagon
18,lagoa,PT,39.05,-27.98,70.30,69,0,6.80,North,Casa Das Faias
83,laguna,US,38.42,-121.42,79.00,50,0,5.01,North,Holiday Inn Express & Suites Elk Grove Central...
103,tupik,RU,54.43,119.93,74.26,54,0,5.17,North,Guest House Orkhideya
173,westport,US,41.14,-73.36,78.01,91,0,8.61,North,The Westport Inn
180,airai,TL,-8.93,125.41,79.41,36,0,2.91,South,Juvinal Martin
196,ribeira grande,PT,38.52,-28.70,70.20,69,0,7.90,North,Quinta da Meia Eira
202,marrakesh,MA,31.63,-8.01,71.60,64,0,3.36,North,Les Jardins de La Koutoubia
215,chardara,KZ,41.25,67.97,73.51,33,0,8.41,North,Жанаргул
261,sao miguel do araguaia,BR,-13.28,-50.16,74.37,37,0,2.35,South,Hotel Executivo Palace


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))